In [11]:

import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.callbacks import get_openai_callback
from tqdm.auto import tqdm
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import shutil
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
import pandas as pd

# Load the Excel file
file_path = '새롬 판정 코드_LLM.xlsx'
df = pd.read_excel(file_path)

# Extract the unique names of "검사명"
unique_names = df['검사명'].unique()

# Display the unique names
unique_names_list = unique_names.tolist()
unique_names_list

['분변 얼리텍 대장암검사',
 '청력검사',
 '혈압검사',
 '안압검사',
 '말초혈액도말검사',
 '위내시경',
 '대장내시경',
 '상복부 초음파',
 '경동맥 초음파',
 '심장 초음파',
 '갑상선 초음파',
 '유방 초음파',
 '하복부 초음파',
 nan,
 '자궁경부 인유두종바이러스',
 '유방 X-선',
 '유방 X-선 + 유방 초음파',
 '흉부 X-선',
 '요추 CT',
 '뇌 CT',
 '요추 X-선',
 '경추 X-선',
 '저선량 폐 CT',
 '복부비만 CT',
 '뇌 MRI',
 '뇌 MRA',
 '경추 MRI',
 '요추 MRI',
 '좌측 무릎 X-선',
 '우측 무릎 X-선',
 '좌측 어깨 X-선',
 '우측 어깨 X-선',
 '경동맥 MRA',
 'C형 간염',
 '폐암검진',
 '자궁경부암',
 '심박 변이도',
 '혈관 경직도 검사',
 '심전도',
 '심장 칼슘 스코어']

In [4]:
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.callbacks import get_openai_callback
from tqdm.auto import tqdm
from dotenv import load_dotenv

load_dotenv()

# Load the Excel file from the specified sheet
filepath = '새롬 판정 코드_LLM.xlsx'
df = pd.read_excel(filepath, sheet_name='소견정보-원본')

df = df.iloc[:586]
df = df.dropna(subset=['검사명'])

# Define and drop columns that are not needed
columns_to_drop = ['카테고리', '질환코드', '판정', '순번', '참조', '공단분류', '장비약어','사용유무','한글소견제목','영문소견제목','영문장비소견','외국어소견','추적관찰내용','구분1','구분2','구분3']
df = df.drop(columns=columns_to_drop)

# Ensure the new columns can accept string values
df['검사명(영문)'] = ''
df['소견명(영문)'] = ''
df['영문종합소견'] = ''

def content_translator(text):
    prompt_template = """
    Translate the content into English. Be aware that the content is medical information or terms. Only give the output.
    
    
    [medical content]
    {medical_content}
    """
    prompt = PromptTemplate.from_template(prompt_template)
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser
    output = chain.invoke({"medical_content": text})

    return output

new_filepath = "새롬 판정 코드_LLM_translated.xlsx"


# Populate new columns using tqdm for progress display
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    test_english = content_translator(row['검사명'])
    opinion_english = content_translator(row['소견명'])
    translated_content = content_translator(row['한글종합소견'])
    
    df.at[index, '검사명(영문)'] = test_english
    df.at[index, '소견명(영문)'] = opinion_english
    df.at[index, '영문종합소견'] = translated_content
    
    if (index + 1) % 100 == 0:
        df.to_excel(new_filepath, index=False)
        print("Excel file has been saved. row :", str(index + 1))

# Save the updated DataFrame to a new Excel file
df.to_excel(new_filepath, index=False)
print("New Excel file has been created:", new_filepath)

  0%|          | 0/442 [00:00<?, ?it/s]

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
import pandas as pd
from langchain_core.documents.base import Document

filepath = "새롬 판정 코드_LLM_translated.xlsx"
df_forloading = pd.read_excel(filepath,sheet_name='subjective')
# Function to create a Document from a DataFrame row
def create_document(row):
    return Document(page_content=row['영문종합소견'], metadata={'test': row['검사명'], 'korean': row['한글종합소견']})

# Sample DataFrame (please replace 'df' with your actual DataFrame)

# Convert each row in the DataFrame to a Document object using tqdm
docs = [create_document(row) for index, row in df_forloading.iterrows()]

print(docs)  # This line is just for verification; remove or replace as needed

[Document(page_content='The gastroscopy results are normal. If you do not have symptoms such as heartburn or indigestion, please undergo a gastroscopy again in 1-2 years.', metadata={'test': '위내시경', 'korean': '* 위내시경 검사 결과 정상입니다. 속쓰림, 소화불량 등의 증상이 없으면 1-2년 후 위내시경 검사를 받으시기 바랍니다.'}), Document(page_content='    * Reflux esophagitis was observed during the gastroscopy. Please avoid alcohol, smoking, overeating, fatty foods, caffeinated beverages, and chocolate, which can exacerbate acid reflux. If you experience symptoms such as heartburn or chest discomfort, please seek medication treatment. It is recommended to undergo regular annual gastroscopy for follow-up observation.', metadata={'test': '위내시경', 'korean': '* 위내시경 검사에서 역류성 식도염이 관찰되었습니다. 위산 역류를 악화시키는 음주, 흡연, 과식, 기름진 음식, 카페인 음료, 초콜릿 등을 피하시고 속쓰림, 흉부 불편감 등의 증상이 있는 경우 약물 치료를 받으시기 바랍니다. 경과 관찰을 위해 매년 정기적인 위내시경 검사를 받으시기 바랍니다.'}), Document(page_content='* The gastroscopy revealed reflux esophagitis and associated sentinel polyps. Avoid alcohol,

In [12]:


FAISS_PATH = 'faiss'

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

if os.path.exists(FAISS_PATH):
        shutil.rmtree(FAISS_PATH)
db = FAISS.from_documents(
        docs, embeddings
    )
db.save_local(FAISS_PATH,index_name ="opinion_results_english")

In [14]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
from langchain.vectorstores import FAISS
db = FAISS.load_local(folder_path=FAISS_PATH,embeddings=embeddings,index_name ="opinion_results_english", allow_dangerous_deserialization=True)
### k를 통해 몇개의 문서를 retrieve 해올지를 정할 수 있습니다.
retriever= db.as_retriever(search_kwargs={'k':5, 'fetch_k':40})
query = """Finding
Reasons for examination : Screening(+)/Problem( -） 
Preparation state : Good(+),Poor( ) 
Sedation : Midazolam (0)mg, Propofol (0)mg


* Esophagus: unremarkable
* Stomach: Atrophic changes on antrum & body
* Duodenum: about 1cm sized polypoid mass at 2nd portion (#1, bx*1)
**Bx : +
**Complication : No 
  
[conclusion]
Chronic atrophic gastritis
r/o duodenal adenoma


[Rec]
f/u EGD 1YL"""
results = retriever.invoke(query)  # Use the search method with 'k' parameter if available
print(f"Number of documents retrieved: {len(results)}")
for doc in results:
    print(doc.metadata['test'])
    
docs_t = db.similarity_search_with_score(query) 
print(f"Number of documents retrieved: {len(docs_t)}")
for doc, score in docs_t:
    print(doc.metadata['test'])
    print(score)

Number of documents retrieved: 5
위내시경
위내시경
위내시경
위내시경
위내시경
Number of documents retrieved: 4
위내시경
0.63050854
위내시경
0.66998684
위내시경
0.67126375
위내시경
0.6800778
